In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

True

In [2]:
# load datasets
from utils.utils import date_range, preprocess_c

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 2, 1),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 2, 1), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X1_train, X2_train, y_train = preprocess_c(df_train)
X1_val, X2_val, y_val = preprocess_c(df_val)

X1_train.shape, X2_train.shape, y_train.shape, X1_val.shape, X2_val.shape, y_val.shape

((1598125, 1, 4),
 (1598125, 30, 4),
 (1598125, 4),
 (263845, 1, 4),
 (263845, 30, 4),
 (263845, 4))

In [3]:
from keras.models import load_model, Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

In [4]:
# retrain view
model_view: Model = load_model('../model/c-view.keras')
cp_view = ModelCheckpoint('../model/c-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit([X1_train, X2_train], y_train[:, 0], validation_data=([X1_val, X2_val], y_val[:, 0]), epochs=3, callbacks=[cp_view])

Epoch 1/3
49942/49942 [==============================] - 130s 3ms/step - loss: 1.0846e-06 - root_mean_squared_error: 0.0010 - val_loss: 6.2182e-06 - val_root_mean_squared_error: 0.0025
Epoch 2/3
49942/49942 [==============================] - 137s 3ms/step - loss: 1.0664e-06 - root_mean_squared_error: 0.0010 - val_loss: 6.2054e-06 - val_root_mean_squared_error: 0.0025
Epoch 3/3
49942/49942 [==============================] - 137s 3ms/step - loss: 1.0462e-06 - root_mean_squared_error: 0.0010 - val_loss: 5.7527e-06 - val_root_mean_squared_error: 0.0024


In [5]:
# retrain cart
model_cart: Model = load_model('../model/c-cart.keras')
cp_cart = ModelCheckpoint('../model/c-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit([X1_train, X2_train], y_train[:, 1], validation_data=([X1_val, X2_val], y_val[:, 0]), epochs=3, callbacks=[cp_cart])

Epoch 1/3
49942/49942 [==============================] - 130s 3ms/step - loss: 1.5385e-06 - root_mean_squared_error: 0.0012 - val_loss: 6.5734e-06 - val_root_mean_squared_error: 0.0026
Epoch 2/3
49942/49942 [==============================] - 127s 3ms/step - loss: 1.2113e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.2182e-06 - val_root_mean_squared_error: 0.0025
Epoch 3/3
49942/49942 [==============================] - 128s 3ms/step - loss: 1.1419e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.0969e-06 - val_root_mean_squared_error: 0.0025


In [6]:
# retrain remove_from_cart
model_remove_from_cart: Model = load_model('../model/c-remove_from_cart.keras')
cp_remove_from_cart = ModelCheckpoint('../model/c-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit([X1_train, X2_train], y_train[:, 2], validation_data=([X1_val, X2_val], y_val[:, 0]), epochs=3, callbacks=[cp_remove_from_cart])

Epoch 1/3
49942/49942 [==============================] - 130s 3ms/step - loss: 1.9319e-06 - root_mean_squared_error: 0.0014 - val_loss: 6.2211e-06 - val_root_mean_squared_error: 0.0025
Epoch 2/3
49942/49942 [==============================] - 120s 2ms/step - loss: 1.1639e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.1053e-06 - val_root_mean_squared_error: 0.0025
Epoch 3/3
49942/49942 [==============================] - 127s 3ms/step - loss: 1.1039e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.0114e-06 - val_root_mean_squared_error: 0.0025


In [7]:
# retrain purchase
model_purchase: Model = load_model('../model/c-purchase.keras')
cp_purchase = ModelCheckpoint('../model/c-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit([X1_train, X2_train], y_train[:, 3], validation_data=([X1_val, X2_val], y_val[:, 0]), epochs=3, callbacks=[cp_purchase])

Epoch 1/3
49942/49942 [==============================] - 130s 3ms/step - loss: 1.8528e-06 - root_mean_squared_error: 0.0014 - val_loss: 6.6739e-06 - val_root_mean_squared_error: 0.0026
Epoch 2/3
49942/49942 [==============================] - 127s 3ms/step - loss: 1.2029e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.2830e-06 - val_root_mean_squared_error: 0.0025
Epoch 3/3
49942/49942 [==============================] - 128s 3ms/step - loss: 1.1256e-06 - root_mean_squared_error: 0.0011 - val_loss: 6.1904e-06 - val_root_mean_squared_error: 0.0025
